In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -qU "langchain[mistralai]"

In [9]:

%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Ensure your VertexAI credentials are configured

from langchain.chat_models import init_chat_model
import vertexai

# Initialize Vertex AI with your project ID and location
vertexai.init(project="your-project-id", location="us-central1")


llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai" ,api_key=api)